<a href="https://colab.research.google.com/github/PratikStar/PratikStar.github.io/blob/master/1_2_Create_Audio_Spectrogram_(Windowed_version)%20(Second%20run).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Pipeline Classes

In [2]:
"""
1- load a file..
2- pad the signal (if necessary)
3- extracting log spectrogram from signal
4- normalise spectrogram
5- save the normalised spectrogram

PreprocessingPipeline
"""
import os
import pickle
import re
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import inspect
import soundfile as sf

SHOW_LOGS = True
LOG_LEVEL = 1
def log(logline, log_level = 1):
    if SHOW_LOGS and log_level <= LOG_LEVEL:
        stack = inspect.stack()
        the_class = stack[1][0].f_locals["self"].__class__.__name__
        the_method = stack[1][0].f_code.co_name
        print(the_class + ": " + logline)

class Loader:
    """Loader is responsible for loading an audio file."""

    def __init__(self, sample_rate, offset, load_duration, window_duration, mono):
        self.sample_rate = sample_rate
        self.load_duration = load_duration
        self.window_duration = window_duration
        self.offset = offset
        self.mono = mono

    def load(self, file_path):
        signal = librosa.load(file_path,
                              sr=self.sample_rate,
                              duration=self.load_duration,
                              offset=self.offset,
                              mono=self.mono)[0]
        log("Shape of the loaded signal: " + str(signal.shape), 2)
        log("Mean of the loaded signal: " + str(np.mean(signal)), 5)
        log("Min of the loaded signal: " + str(np.amin(signal)), 5)
        log("Max of the loaded signal: " + str(np.amax(signal)), 5)
        log("Raw signal: " + str(signal), 5)
        return signal


class Padder:
    """Padder is responsible to apply padding to an array."""

    def __init__(self, mode="constant"):
        self.mode = mode

    def left_pad(self, array, num_missing_items):
        padded_array = np.pad(array,
                              (num_missing_items, 0),
                              mode=self.mode)
        return padded_array

    def right_pad(self, array, num_missing_items):
        padded_array = np.pad(array,
                              (0, num_missing_items),
                              mode=self.mode)
        return padded_array


class LogSpectrogramExtractor:
    """LogSpectrogramExtractor extracts log spectrograms (in dB) from a
    time-series signal.
    """

    def __init__(self, frame_size, hop_length):
        self.frame_size = frame_size
        self.hop_length = hop_length

    def extract(self, signal):
        stft = librosa.stft(signal,
                            n_fft=self.frame_size,
                            hop_length=self.hop_length)[:-1]
        log("Shape of stft: " + str(stft.shape), 2)
        spectrogram = np.abs(stft)
        log_spectrogram = librosa.amplitude_to_db(spectrogram)
        return log_spectrogram

class FeatureExtractor:
    """ Extracts the required features"""

    def extract_ampid_from_name(self, filename):
        amp_id = int(filename.split('-')[0])
        log("Amp ID: " + str(amp_id), 5)
        return amp_id
    def extract_clipid_from_name(self, filename):
        clip_id = int(filename.split('-')[0])
        log("Clip ID: " + str(clip_id), 5)
        return clip_id

class MinMaxNormaliser:
    """MinMaxNormaliser applies min max normalisation to an array."""

    def __init__(self, min_val, max_val):
        self.min = min_val
        self.max = max_val

    def normalise(self, array):

        norm_array = (array - array.min()) / (array.max() - array.min())
        norm_array = norm_array * (self.max - self.min) + self.min
        log("Shape of norm_array: " + str(norm_array.shape), 2)
        log("Max of norm_array: " + str((array.max())), 5)
        log("Min of norm_array: " + str((array.min())), 5)
        return norm_array

    def denormalise(self, norm_array, original_min, original_max):
        array = (norm_array - self.min) / (self.max - self.min)
        array = array * (original_max - original_min) + original_min
        return array


class Saver:
    """saver is responsible to save features, and the min max values."""

    def __init__(self, feature_save_dir, min_max_values_save_dir):
        self.feature_save_dir = feature_save_dir
        Utils._create_folder_if_it_doesnt_exist(self.feature_save_dir)
        self.min_max_values_save_dir = min_max_values_save_dir

    def save_feature(self, feature, file_name):
        save_path = self._generate_save_path(file_name)
        np.save(save_path, feature)

    def save_min_max_values(self, min_max_values):
        save_path = os.path.join(self.min_max_values_save_dir,
                                 "min_max_values.pkl")
        self._save(min_max_values, save_path)

    @staticmethod
    def _save(data, save_path):
        with open(save_path, "wb") as f:
            pickle.dump(data, f)

    def _generate_save_path(self, file_name):
        save_path = os.path.join(self.feature_save_dir, file_name + ".npy")
        return save_path

class Visualizer:
    def __init__(self, file_dir, frame_size, hop_length, offset):
        self.file_dir = file_dir
        self.frame_size = frame_size
        self.hop_length = hop_length
        self.offset = offset
        pass
    
    def visualize(self, spectrogram, file_name):
        plt.ioff()
        fig, ax = plt.subplots()
        try:
            img = librosa.display.specshow(spectrogram, y_axis='log', x_axis='time', ax=ax)
        except IndexError as e:
            log("Null spectrogram for file: " + file_name, 1)
            return
        ax.set_title("Frame Size: " + str(self.frame_size) + ", Hop Length: " + str(self.hop_length) + ", Offset: " + str(self.offset))
        fig.colorbar(img, ax=ax, format="%+2.0f dB")
        Utils._create_folder_if_it_doesnt_exist(self.file_dir)
        fig.savefig(os.path.join(self.file_dir, file_name + ".png"))
        plt.close(fig)

class Utils:
    def __init__(self):
        pass
    @staticmethod
    def _create_folder_if_it_doesnt_exist(folder):
        if not os.path.exists(folder):
            os.makedirs(folder)

class AudioReconstructor:
    """Processes stft inverse and saves the audio file"""
    def __init__(self, file_dir, hop_length, frame_size):
        self.file_dir = file_dir
        self.hop_length = hop_length
        self.frame_size = frame_size

    def reconstruct(self, features, file_name):
        # Invert using Griffin-Lim
        log("Shape of features: " + str(features.shape), 1)
        features_inv = librosa.griffinlim(features,
                                          hop_length=self.hop_length,
                                          win_length=self.frame_size)
        log("Shape of Inverse features: " + str(features_inv.shape), 1)
        Utils._create_folder_if_it_doesnt_exist(self.file_dir)
        sf.write(os.path.join(self.file_dir, file_name ), y_inv, 22050)
        # save audio
    def reconstruct_from_path(self, file_path):
        # Invert using Griffin-Lim
        features = np.load(file_path)
        self.reconstruct(features, os.path.basename(file_path))
    
class PreprocessingPipeline:
    """PreprocessingPipeline processes a single audio file, applying
    the following steps to each file:
        1- load a file
        2- pad the signal (if necessary)
        3- extracting log spectrogram from signal
        4- normalise spectrogram
        5- save the normalised spectrogram

    Storing the min max values for all the log spectrograms.
    """

    def __init__(self):
        self.padder = None
        self.extractor = None
        self.feature_extractor = None
        self.normaliser = None
        self.saver = None
        self.visualizer = None
        self.reconstructor = None
        self.min_max_values = {}
        self._loader = None
        self._num_expected_samples = None

    @property
    def loader(self):
        return self._loader

    @loader.setter
    def loader(self, loader):
        self._loader = loader
        self._num_expected_samples = int(loader.sample_rate * loader.window_duration)

    def process_file(self, audio_files_dir, audio_file_name, window_id, save_flg=True, visualize_flg=True, external_features_flg=False):

        # self._process_file(root, file, save_flg, visualize_flg, external_features_flg)

        file_path = os.path.join(audio_files_dir, audio_file_name)
        signal = self.loader.load(file_path)
        if external_features_flg:
            external_features = [ self.feature_extractor.extract_ampid_from_name(audio_file_name),
                                  self.feature_extractor.extract_clipid_from_name(audio_file_name),
                                  window_id                         
                                ]
            signal = np.concatenate((external_features, signal))

        if self._is_padding_necessary(signal):
            signal = self._apply_padding(signal)
        feature = self.extractor.extract(signal)

        norm_feature = self.normaliser.normalise(feature)
        name_split = audio_file_name.split(" ")
        file_name = name_split[0] + "-" + ('%02d' % (window_id)) + " " + " ".join(name_split[1:])
        print("\t" + file_name)
        if visualize_flg:
            self.visualizer.visualize(norm_feature, file_name)

        if save_flg:
            save_path = self.saver.save_feature(norm_feature, file_name)
            # self._store_min_max_value(save_path, feature.min(), feature.max())
        log(f"Processed file {audio_file_name}", 5)

        # self.saver.save_min_max_values(self.min_max_values)

    def _is_padding_necessary(self, signal):
        if len(signal) < self._num_expected_samples:
            log("Padding necessary", 5)
            log("Signal: " + str(len(signal)), 5)
            log("Exp samples: " + str(self._num_expected_samples), 5)
            return True
        return False

    def _apply_padding(self, signal):
        num_missing_samples = self._num_expected_samples - len(signal)
        padded_signal = self.padder.right_pad(signal, num_missing_samples)
        return padded_signal

    def _store_min_max_value(self, save_path, min_val, max_val):
        self.min_max_values[save_path] = {
            "min": min_val,
            "max": max_val
        }


## Driver Code

In [3]:
## TOBE <AMP_ID>-<CLIP_ID>-<WINDOW_ID> <AMP_NAME>.wav.npy
## 00000-06-00 DI.wav.png is not present

FRAME_SIZE = 512
HOP_LENGTH = FRAME_SIZE // 2 # smaller better?
OFFSET = 0
WINDOW_DURATION = 0.74  # in seconds
SAMPLE_RATE = 22050
MONO = True

# Dimensions: (FRAME_SIZE / 2) x ceil(SAMPLE_RATE * DURATION / HOP_LENGTH)

## SOURCE
AUDIO_FILES_DIR = "/content/drive/MyDrive/Music/VAE/audio/"
AUDIO_REGEX = "^(?!00000.*$).*\.wav$"

## OUTPUT
SPECTROGRAMS_SAVE_DIR = "/content/drive/MyDrive/Music/VAE/ICASSP/spectrogram/"
SPECTROGRAMS_IMG_SAVE_DIR = "/content/drive/MyDrive/Music/VAE/ICASSP/spectrogram-img/"
RECONSTRUCTED_AUDIO_SAVE_DIR = "/content/drive/MyDrive/Music/VAE/ICASSP/audio-reconstructed/"
MIN_MAX_VALUES_SAVE_DIR = "/content/drive/MyDrive/Music/VAE/ICASSP"

NUMBER_OF_CLIPS = 10000000

# instantiate all objects
# loader = Loader(SAMPLE_RATE, OFFSET, WINDOW_DURATION, MONO)
padder = Padder()
log_spectrogram_extractor = LogSpectrogramExtractor(FRAME_SIZE, HOP_LENGTH)
feature_extractor = FeatureExtractor()
min_max_normaliser = MinMaxNormaliser(0, 1)
saver = Saver(SPECTROGRAMS_SAVE_DIR, MIN_MAX_VALUES_SAVE_DIR)
visualizer = Visualizer(SPECTROGRAMS_IMG_SAVE_DIR, FRAME_SIZE, HOP_LENGTH, OFFSET)
reconstructor = AudioReconstructor(RECONSTRUCTED_AUDIO_SAVE_DIR, HOP_LENGTH, FRAME_SIZE)

preprocessing_pipeline = PreprocessingPipeline()
# preprocessing_pipeline.loader = loader
preprocessing_pipeline.padder = padder
preprocessing_pipeline.extractor = log_spectrogram_extractor
preprocessing_pipeline.feature_extractor = feature_extractor
preprocessing_pipeline.normaliser = min_max_normaliser
preprocessing_pipeline.saver = saver
preprocessing_pipeline.reconstructor = reconstructor

number_of_clips = NUMBER_OF_CLIPS
# for root, _, files in os.walk(AUDIO_FILES_DIR):
#     for file in files:

#         if re.match(AUDIO_REGEX, file) and number_of_clips > 0:
#             print(file)
#             number_of_clips -= 1
#             y, sr = librosa.load(os.path.join(AUDIO_FILES_DIR, file), sr=SAMPLE_RATE, mono=MONO)
#             duration = librosa.get_duration(y=y, sr=sr)
#             window_id = 0
#             while True:
#                 if (window_id) * WINDOW_DURATION >= duration:
#                     break
                
#                 offset = window_id * WINDOW_DURATION
#                 loader = Loader(SAMPLE_RATE, offset, min(WINDOW_DURATION, duration - offset), WINDOW_DURATION, MONO)
#                 preprocessing_pipeline.loader = loader
#                 visualizer = Visualizer(SPECTROGRAMS_IMG_SAVE_DIR, FRAME_SIZE, HOP_LENGTH, offset)
#                 preprocessing_pipeline.visualizer = visualizer

#                 preprocessing_pipeline.process_file(AUDIO_FILES_DIR, file, window_id, save_flg=True, visualize_flg=True, external_features_flg=False)
#                 window_id += 1



In [4]:
files = ['00017-07 05A Placater Clean.wav', '00017-06 05A Placater Clean.wav', '00017-12 05A Placater Clean.wav', '00017-09 05A Placater Clean.wav', '00017-08 05A Placater Clean.wav', '00017-01 05A Placater Clean.wav', '00017-10 05A Placater Clean.wav', '00017-02 05A Placater Clean.wav', '00017-04 05A Placater Clean.wav', '00018-04 05B Interstate Zed.wav', '00018-10 05B Interstate Zed.wav', '00018-02 05B Interstate Zed.wav', '00018-08 05B Interstate Zed.wav', '00018-03 05B Interstate Zed.wav', '00018-01 05B Interstate Zed.wav', '00018-05 05B Interstate Zed.wav', '00018-09 05B Interstate Zed.wav', '00018-07 05B Interstate Zed.wav', '00018-11 05B Interstate Zed.wav', '00018-12 05B Interstate Zed.wav', '00018-06 05B Interstate Zed.wav', '00019-01 05C Placater Dirty.wav', '00019-11 05C Placater Dirty.wav', '00019-05 05C Placater Dirty.wav', '00019-12 05C Placater Dirty.wav', '00019-06 05C Placater Dirty.wav', '00019-03 05C Placater Dirty.wav', '00019-02 05C Placater Dirty.wav', '00019-04 05C Placater Dirty.wav', '00019-09 05C Placater Dirty.wav', '00019-07 05C Placater Dirty.wav', '00019-08 05C Placater Dirty.wav', '00019-10 05C Placater Dirty.wav', '00020-04 05D PV Panama.wav', '00020-05 05D PV Panama.wav', '00020-09 05D PV Panama.wav', '00020-06 05D PV Panama.wav', '00020-03 05D PV Panama.wav', '00020-12 05D PV Panama.wav', '00020-11 05D PV Panama.wav', '00020-02 05D PV Panama.wav', '00020-08 05D PV Panama.wav', '00020-07 05D PV Panama.wav', '00020-01 05D PV Panama.wav', '00020-10 05D PV Panama.wav', '00021-05 06A Cali Texas Ch 1.wav', '00021-07 06A Cali Texas Ch 1.wav', '00021-01 06A Cali Texas Ch 1.wav', '00021-10 06A Cali Texas Ch 1.wav', '00021-08 06A Cali Texas Ch 1.wav', '00021-06 06A Cali Texas Ch 1.wav', '00021-09 06A Cali Texas Ch 1.wav', '00021-04 06A Cali Texas Ch 1.wav', '00021-11 06A Cali Texas Ch 1.wav', '00021-02 06A Cali Texas Ch 1.wav', '00021-12 06A Cali Texas Ch 1.wav', '00021-03 06A Cali Texas Ch 1.wav', '00022-07 06B Essex A15.wav', '00022-06 06B Essex A15.wav', '00022-10 06B Essex A15.wav', '00022-02 06B Essex A15.wav', '00022-08 06B Essex A15.wav', '00022-03 06B Essex A15.wav', '00022-01 06B Essex A15.wav', '00022-09 06B Essex A15.wav', '00022-05 06B Essex A15.wav', '00022-11 06B Essex A15.wav', '00022-12 06B Essex A15.wav', '00022-04 06B Essex A15.wav', '00023-08 06C Derailed Ingrid.wav', '00023-04 06C Derailed Ingrid.wav', '00023-01 06C Derailed Ingrid.wav', '00023-06 06C Derailed Ingrid.wav', '00023-03 06C Derailed Ingrid.wav', '00023-09 06C Derailed Ingrid.wav', '00023-02 06C Derailed Ingrid.wav', '00023-07 06C Derailed Ingrid.wav', '00023-12 06C Derailed Ingrid.wav', '00023-11 06C Derailed Ingrid.wav', '00023-10 06C Derailed Ingrid.wav', '00023-05 06C Derailed Ingrid.wav', '00024-06 06D German Mahadeva.wav', '00024-11 06D German Mahadeva.wav', '00024-03 06D German Mahadeva.wav', '00024-01 06D German Mahadeva.wav', '00024-07 06D German Mahadeva.wav', '00024-02 06D German Mahadeva.wav', '00024-05 06D German Mahadeva.wav', '00024-10 06D German Mahadeva.wav', '00024-12 06D German Mahadeva.wav', '00024-09 06D German Mahadeva.wav', '00024-08 06D German Mahadeva.wav', '00024-04 06D German Mahadeva.wav', '00025-10 07A WhoWatt 100.wav', '00025-12 07A WhoWatt 100.wav', '00025-04 07A WhoWatt 100.wav', '00025-07 07A WhoWatt 100.wav', '00025-02 07A WhoWatt 100.wav', '00025-11 07A WhoWatt 100.wav', '00025-01 07A WhoWatt 100.wav', '00025-08 07A WhoWatt 100.wav', '00025-05 07A WhoWatt 100.wav', '00025-03 07A WhoWatt 100.wav', '00025-09 07A WhoWatt 100.wav', '00025-06 07A WhoWatt 100.wav', '00026-12 07B Line 6 Litigator.wav', '00026-08 07B Line 6 Litigator.wav', '00026-06 07B Line 6 Litigator.wav', '00026-02 07B Line 6 Litigator.wav', '00026-04 07B Line 6 Litigator.wav', '00026-10 07B Line 6 Litigator.wav', '00026-05 07B Line 6 Litigator.wav', '00026-09 07B Line 6 Litigator.wav', '00026-11 07B Line 6 Litigator.wav', '00026-03 07B Line 6 Litigator.wav', '00026-07 07B Line 6 Litigator.wav', '00026-01 07B Line 6 Litigator.wav', '00027-03 07C Cartographer.wav', '00027-11 07C Cartographer.wav', '00027-09 07C Cartographer.wav', '00027-01 07C Cartographer.wav', '00027-06 07C Cartographer.wav', '00027-07 07C Cartographer.wav', '00027-10 07C Cartographer.wav', '00027-08 07C Cartographer.wav', '00027-04 07C Cartographer.wav', '00027-05 07C Cartographer.wav', '00027-12 07C Cartographer.wav', '00027-02 07C Cartographer.wav', '00028-04 07D ANGL Meteor.wav', '00028-06 07D ANGL Meteor.wav', '00028-08 07D ANGL Meteor.wav', '00028-05 07D ANGL Meteor.wav', '00028-02 07D ANGL Meteor.wav', '00028-03 07D ANGL Meteor.wav', '00028-12 07D ANGL Meteor.wav', '00028-10 07D ANGL Meteor.wav', '00028-01 07D ANGL Meteor.wav', '00028-11 07D ANGL Meteor.wav', '00028-07 07D ANGL Meteor.wav', '00028-09 07D ANGL Meteor.wav', '00029-03 08A US Small Tweed.wav', '00029-05 08A US Small Tweed.wav', '00029-01 08A US Small Tweed.wav', '00029-04 08A US Small Tweed.wav', '00029-10 08A US Small Tweed.wav', '00029-08 08A US Small Tweed.wav', '00029-02 08A US Small Tweed.wav', '00029-06 08A US Small Tweed.wav', '00029-09 08A US Small Tweed.wav', '00029-12 08A US Small Tweed.wav', '00029-07 08A US Small Tweed.wav', '00029-11 08A US Small Tweed.wav', '00030-06 08B Divided Duo.wav', '00030-07 08B Divided Duo.wav', '00030-01 08B Divided Duo.wav', '00030-03 08B Divided Duo.wav', '00030-04 08B Divided Duo.wav', '00030-09 08B Divided Duo.wav', '00030-11 08B Divided Duo.wav', '00030-02 08B Divided Duo.wav', '00030-08 08B Divided Duo.wav', '00030-05 08B Divided Duo.wav', '00030-10 08B Divided Duo.wav', '00030-12 08B Divided Duo.wav', '00031-04 08C Brit P75 Brt.wav', '00031-12 08C Brit P75 Brt.wav', '00031-08 08C Brit P75 Brt.wav', '00031-03 08C Brit P75 Brt.wav', '00031-09 08C Brit P75 Brt.wav', '00031-07 08C Brit P75 Brt.wav', '00031-10 08C Brit P75 Brt.wav', '00031-05 08C Brit P75 Brt.wav', '00031-11 08C Brit P75 Brt.wav', '00031-02 08C Brit P75 Brt.wav', '00031-06 08C Brit P75 Brt.wav', '00031-01 08C Brit P75 Brt.wav', '00032-05 08D Line 6 Badonk.wav', '00032-07 08D Line 6 Badonk.wav', '00032-10 08D Line 6 Badonk.wav', '00032-06 08D Line 6 Badonk.wav', '00032-11 08D Line 6 Badonk.wav', '00032-03 08D Line 6 Badonk.wav', '00032-08 08D Line 6 Badonk.wav', '00032-01 08D Line 6 Badonk.wav', '00032-09 08D Line 6 Badonk.wav', '00032-02 08D Line 6 Badonk.wav', '00032-12 08D Line 6 Badonk.wav', '00032-04 08D Line 6 Badonk.wav', '00033-04 09A DI.wav', '00033-08 09A DI.wav', '00033-05 09A DI.wav', '00033-07 09A DI.wav', '00033-02 09A DI.wav', '00033-06 09A DI.wav', '00033-10 09A DI.wav', '00033-03 09A DI.wav', '00033-01 09A DI.wav', '00033-11 09A DI.wav', '00033-09 09A DI.wav', '00033-12 09A DI.wav', '00034-06 09B BAS_SVT-4 Pro.wav', '00034-05 09B BAS_SVT-4 Pro.wav', '00034-01 09B BAS_SVT-4 Pro.wav', '00034-02 09B BAS_SVT-4 Pro.wav', '00034-08 09B BAS_SVT-4 Pro.wav', '00034-03 09B BAS_SVT-4 Pro.wav', '00034-09 09B BAS_SVT-4 Pro.wav', '00034-04 09B BAS_SVT-4 Pro.wav', '00034-11 09B BAS_SVT-4 Pro.wav', '00034-12 09B BAS_SVT-4 Pro.wav', '00034-07 09B BAS_SVT-4 Pro.wav', '00034-10 09B BAS_SVT-4 Pro.wav', '00035-06 09C BAS_Cali Bass.wav', '00035-10 09C BAS_Cali Bass.wav', '00035-11 09C BAS_Cali Bass.wav', '00035-01 09C BAS_Cali Bass.wav', '00035-09 09C BAS_Cali Bass.wav', '00035-04 09C BAS_Cali Bass.wav', '00035-12 09C BAS_Cali Bass.wav', '00035-07 09C BAS_Cali Bass.wav', '00035-05 09C BAS_Cali Bass.wav', '00035-08 09C BAS_Cali Bass.wav', '00035-03 09C BAS_Cali Bass.wav', '00035-02 09C BAS_Cali Bass.wav', '00036-06 09D BAS_Aqua 51.wav', '00036-11 09D BAS_Aqua 51.wav', '00036-04 09D BAS_Aqua 51.wav', '00036-08 09D BAS_Aqua 51.wav', '00036-09 09D BAS_Aqua 51.wav', '00036-12 09D BAS_Aqua 51.wav', '00036-10 09D BAS_Aqua 51.wav', '00036-03 09D BAS_Aqua 51.wav', '00036-01 09D BAS_Aqua 51.wav', '00036-05 09D BAS_Aqua 51.wav', '00036-02 09D BAS_Aqua 51.wav', '00036-07 09D BAS_Aqua 51.wav', '00037-05 10A BAS_Cougar 800.wav', '00037-08 10A BAS_Cougar 800.wav', '00037-11 10A BAS_Cougar 800.wav', '00037-06 10A BAS_Cougar 800.wav', '00037-03 10A BAS_Cougar 800.wav', '00037-10 10A BAS_Cougar 800.wav', '00037-02 10A BAS_Cougar 800.wav', '00037-12 10A BAS_Cougar 800.wav', '00037-09 10A BAS_Cougar 800.wav', '00037-07 10A BAS_Cougar 800.wav', '00037-04 10A BAS_Cougar 800.wav', '00037-01 10A BAS_Cougar 800.wav', '00038-11 10B BAS_SVT Nrm.wav', '00038-05 10B BAS_SVT Nrm.wav', '00038-10 10B BAS_SVT Nrm.wav', '00038-04 10B BAS_SVT Nrm.wav', '00038-08 10B BAS_SVT Nrm.wav', '00038-12 10B BAS_SVT Nrm.wav', '00038-03 10B BAS_SVT Nrm.wav', '00038-01 10B BAS_SVT Nrm.wav', '00038-02 10B BAS_SVT Nrm.wav', '00038-07 10B BAS_SVT Nrm.wav', '00038-09 10B BAS_SVT Nrm.wav', '00038-06 10B BAS_SVT Nrm.wav', '00039-04 10C BAS_Cali 400 Ch1.wav', '00039-09 10C BAS_Cali 400 Ch1.wav', '00039-05 10C BAS_Cali 400 Ch1.wav', '00039-12 10C BAS_Cali 400 Ch1.wav', '00039-07 10C BAS_Cali 400 Ch1.wav', '00039-10 10C BAS_Cali 400 Ch1.wav', '00039-02 10C BAS_Cali 400 Ch1.wav', '00039-01 10C BAS_Cali 400 Ch1.wav', '00039-08 10C BAS_Cali 400 Ch1.wav', '00039-06 10C BAS_Cali 400 Ch1.wav', '00039-03 10C BAS_Cali 400 Ch1.wav', '00039-11 10C BAS_Cali 400 Ch1.wav', '00040-06 10D BAS_Del Sol 300.wav', '00040-05 10D BAS_Del Sol 300.wav', '00040-04 10D BAS_Del Sol 300.wav', '00040-01 10D BAS_Del Sol 300.wav', '00040-02 10D BAS_Del Sol 300.wav', '00040-03 10D BAS_Del Sol 300.wav', '00040-10 10D BAS_Del Sol 300.wav', '00040-08 10D BAS_Del Sol 300.wav', '00040-11 10D BAS_Del Sol 300.wav', '00040-12 10D BAS_Del Sol 300.wav', '00040-09 10D BAS_Del Sol 300.wav', '00040-07 10D BAS_Del Sol 300.wav', '00041-10 11A BAS_Woody Blue.wav', '00041-09 11A BAS_Woody Blue.wav', '00041-12 11A BAS_Woody Blue.wav', '00041-01 11A BAS_Woody Blue.wav', '00041-05 11A BAS_Woody Blue.wav', '00041-06 11A BAS_Woody Blue.wav', '00041-11 11A BAS_Woody Blue.wav', '00041-03 11A BAS_Woody Blue.wav', '00041-02 11A BAS_Woody Blue.wav', '00041-04 11A BAS_Woody Blue.wav', '00041-08 11A BAS_Woody Blue.wav', '00041-07 11A BAS_Woody Blue.wav', '00042-08 11B Trademark.wav', '00042-01 11B Trademark.wav', '00042-05 11B Trademark.wav', '00042-07 11B Trademark.wav', '00042-10 11B Trademark.wav', '00042-11 11B Trademark.wav', '00042-09 11B Trademark.wav', '00042-12 11B Trademark.wav', '00042-03 11B Trademark.wav', '00042-02 11B Trademark.wav', '00042-06 11B Trademark.wav', '00042-04 11B Trademark.wav', '00043-12 11C AUS Flood.wav', '00043-04 11C AUS Flood.wav', '00043-09 11C AUS Flood.wav', '00043-01 11C AUS Flood.wav', '00043-05 11C AUS Flood.wav', '00043-06 11C AUS Flood.wav', '00043-08 11C AUS Flood.wav', '00043-11 11C AUS Flood.wav', '00043-10 11C AUS Flood.wav', '00043-03 11C AUS Flood.wav', '00043-07 11C AUS Flood.wav', '00043-02 11C AUS Flood.wav', "00044-09 11D Justice Fo Y'all.wav", "00044-10 11D Justice Fo Y'all.wav", "00044-11 11D Justice Fo Y'all.wav", "00044-05 11D Justice Fo Y'all.wav", "00044-01 11D Justice Fo Y'all.wav", "00044-03 11D Justice Fo Y'all.wav", "00044-12 11D Justice Fo Y'all.wav", "00044-06 11D Justice Fo Y'all.wav", "00044-04 11D Justice Fo Y'all.wav", "00044-07 11D Justice Fo Y'all.wav", "00044-08 11D Justice Fo Y'all.wav", "00044-02 11D Justice Fo Y'all.wav", '00045-08 12A Lonely Hearts.wav', '00045-10 12A Lonely Hearts.wav', '00045-04 12A Lonely Hearts.wav', '00045-03 12A Lonely Hearts.wav', '00045-12 12A Lonely Hearts.wav', '00045-11 12A Lonely Hearts.wav', '00045-05 12A Lonely Hearts.wav', '00045-01 12A Lonely Hearts.wav', '00045-09 12A Lonely Hearts.wav', '00045-06 12A Lonely Hearts.wav', '00045-02 12A Lonely Hearts.wav', '00045-07 12A Lonely Hearts.wav', '00046-10 12B Pull Me Under.wav', '00046-11 12B Pull Me Under.wav', '00046-07 12B Pull Me Under.wav', '00046-12 12B Pull Me Under.wav', '00046-05 12B Pull Me Under.wav', '00046-03 12B Pull Me Under.wav', '00046-01 12B Pull Me Under.wav', '00046-09 12B Pull Me Under.wav', '00046-04 12B Pull Me Under.wav', '00046-02 12B Pull Me Under.wav', '00046-08 12B Pull Me Under.wav', '00046-06 12B Pull Me Under.wav', '00047-08 12C Stone Cold Loco.wav', '00047-10 12C Stone Cold Loco.wav', '00047-09 12C Stone Cold Loco.wav', '00047-04 12C Stone Cold Loco.wav', '00047-03 12C Stone Cold Loco.wav', '00047-06 12C Stone Cold Loco.wav', '00047-07 12C Stone Cold Loco.wav', '00047-12 12C Stone Cold Loco.wav', '00047-01 12C Stone Cold Loco.wav', '00047-11 12C Stone Cold Loco.wav', '00047-05 12C Stone Cold Loco.wav', '00047-02 12C Stone Cold Loco.wav', '00048-12 12D Plush Garden.wav', '00048-05 12D Plush Garden.wav', '00048-10 12D Plush Garden.wav', '00048-08 12D Plush Garden.wav', '00048-07 12D Plush Garden.wav', '00048-11 12D Plush Garden.wav', '00048-09 12D Plush Garden.wav', '00048-06 12D Plush Garden.wav', '00048-02 12D Plush Garden.wav', '00048-01 12D Plush Garden.wav', '00048-03 12D Plush Garden.wav', '00048-04 12D Plush Garden.wav', '00049-02 13A Cowboys from DFW.wav', '00049-04 13A Cowboys from DFW.wav', '00049-09 13A Cowboys from DFW.wav', '00049-11 13A Cowboys from DFW.wav', '00049-03 13A Cowboys from DFW.wav', '00049-12 13A Cowboys from DFW.wav', '00049-08 13A Cowboys from DFW.wav', '00049-10 13A Cowboys from DFW.wav', '00049-06 13A Cowboys from DFW.wav', '00049-01 13A Cowboys from DFW.wav', '00049-05 13A Cowboys from DFW.wav', '00049-07 13A Cowboys from DFW.wav', '00050-06 13B G.O.A.T Rodeo.wav', '00050-03 13B G.O.A.T Rodeo.wav', '00050-12 13B G.O.A.T Rodeo.wav', '00050-10 13B G.O.A.T Rodeo.wav', '00050-11 13B G.O.A.T Rodeo.wav', '00050-01 13B G.O.A.T Rodeo.wav', '00050-04 13B G.O.A.T Rodeo.wav', '00050-05 13B G.O.A.T Rodeo.wav', '00050-02 13B G.O.A.T Rodeo.wav', '00050-07 13B G.O.A.T Rodeo.wav', '00050-09 13B G.O.A.T Rodeo.wav', '00050-08 13B G.O.A.T Rodeo.wav', '00051-03 13C BIG DUBB.wav', '00051-01 13C BIG DUBB.wav', '00051-05 13C BIG DUBB.wav', '00051-04 13C BIG DUBB.wav', '00051-02 13C BIG DUBB.wav', '00051-09 13C BIG DUBB.wav', '00051-11 13C BIG DUBB.wav', '00051-10 13C BIG DUBB.wav', '00051-12 13C BIG DUBB.wav', '00051-07 13C BIG DUBB.wav', '00051-06 13C BIG DUBB.wav', '00051-08 13C BIG DUBB.wav', '00052-02 13D BIG VENUE DRIVE.wav', '00052-10 13D BIG VENUE DRIVE.wav', '00052-04 13D BIG VENUE DRIVE.wav', '00052-09 13D BIG VENUE DRIVE.wav', '00052-11 13D BIG VENUE DRIVE.wav', '00052-12 13D BIG VENUE DRIVE.wav', '00052-01 13D BIG VENUE DRIVE.wav', '00052-08 13D BIG VENUE DRIVE.wav', '00052-07 13D BIG VENUE DRIVE.wav', '00052-06 13D BIG VENUE DRIVE.wav', '00052-05 13D BIG VENUE DRIVE.wav', '00052-03 13D BIG VENUE DRIVE.wav', '00053-01 14A BUBBLE NEST.wav', '00053-09 14A BUBBLE NEST.wav', '00053-02 14A BUBBLE NEST.wav', '00053-07 14A BUBBLE NEST.wav', '00053-03 14A BUBBLE NEST.wav', '00053-12 14A BUBBLE NEST.wav', '00053-06 14A BUBBLE NEST.wav', '00053-08 14A BUBBLE NEST.wav', '00053-05 14A BUBBLE NEST.wav', '00053-04 14A BUBBLE NEST.wav', '00053-11 14A BUBBLE NEST.wav', '00053-10 14A BUBBLE NEST.wav', '00054-10 14B DUSTED.wav', '00054-01 14B DUSTED.wav', '00054-09 14B DUSTED.wav', '00054-06 14B DUSTED.wav', '00054-02 14B DUSTED.wav', '00054-04 14B DUSTED.wav', '00054-11 14B DUSTED.wav', '00054-12 14B DUSTED.wav', '00054-07 14B DUSTED.wav', '00054-08 14B DUSTED.wav', '00054-05 14B DUSTED.wav', '00054-03 14B DUSTED.wav', '00055-10 14C SUNRISE DRIVE.wav', '00055-05 14C SUNRISE DRIVE.wav', '00055-11 14C SUNRISE DRIVE.wav', '00055-08 14C SUNRISE DRIVE.wav', '00055-03 14C SUNRISE DRIVE.wav', '00055-09 14C SUNRISE DRIVE.wav', '00055-01 14C SUNRISE DRIVE.wav', '00055-07 14C SUNRISE DRIVE.wav', '00055-04 14C SUNRISE DRIVE.wav', '00055-02 14C SUNRISE DRIVE.wav', '00055-12 14C SUNRISE DRIVE.wav', '00055-06 14C SUNRISE DRIVE.wav', '00056-03 14D GLISTEN.wav', '00056-09 14D GLISTEN.wav', '00056-07 14D GLISTEN.wav', '00056-05 14D GLISTEN.wav', '00056-11 14D GLISTEN.wav', '00056-01 14D GLISTEN.wav', '00056-02 14D GLISTEN.wav', '00056-12 14D GLISTEN.wav', '00056-04 14D GLISTEN.wav', '00056-06 14D GLISTEN.wav', '00056-08 14D GLISTEN.wav', '00056-10 14D GLISTEN.wav', '00057-07 15A WATERS IN HELL.wav', '00057-03 15A WATERS IN HELL.wav', '00057-12 15A WATERS IN HELL.wav', '00057-04 15A WATERS IN HELL.wav', '00057-06 15A WATERS IN HELL.wav', '00057-02 15A WATERS IN HELL.wav', '00057-01 15A WATERS IN HELL.wav', '00057-08 15A WATERS IN HELL.wav', '00057-10 15A WATERS IN HELL.wav', '00057-11 15A WATERS IN HELL.wav', '00057-09 15A WATERS IN HELL.wav', '00057-05 15A WATERS IN HELL.wav', '00058-05 15B FAUX 7 STG CHUG.wav', '00058-12 15B FAUX 7 STG CHUG.wav', '00058-03 15B FAUX 7 STG CHUG.wav', '00058-02 15B FAUX 7 STG CHUG.wav', '00058-08 15B FAUX 7 STG CHUG.wav', '00058-09 15B FAUX 7 STG CHUG.wav', '00058-11 15B FAUX 7 STG CHUG.wav', '00058-01 15B FAUX 7 STG CHUG.wav', '00058-07 15B FAUX 7 STG CHUG.wav', '00058-06 15B FAUX 7 STG CHUG.wav', '00058-10 15B FAUX 7 STG CHUG.wav', '00058-04 15B FAUX 7 STG CHUG.wav', '00059-04 15C RICHEESE.wav', '00059-10 15C RICHEESE.wav', '00059-08 15C RICHEESE.wav', '00059-05 15C RICHEESE.wav', '00059-07 15C RICHEESE.wav', '00059-09 15C RICHEESE.wav', '00059-06 15C RICHEESE.wav', '00059-12 15C RICHEESE.wav', '00059-02 15C RICHEESE.wav', '00059-03 15C RICHEESE.wav', '00059-01 15C RICHEESE.wav', '00059-11 15C RICHEESE.wav', '00060-06 15D RC REINCARNATION.wav', '00060-02 15D RC REINCARNATION.wav', '00060-08 15D RC REINCARNATION.wav', '00060-01 15D RC REINCARNATION.wav', '00060-04 15D RC REINCARNATION.wav', '00060-07 15D RC REINCARNATION.wav', '00060-03 15D RC REINCARNATION.wav', '00060-05 15D RC REINCARNATION.wav', '00060-10 15D RC REINCARNATION.wav', '00060-11 15D RC REINCARNATION.wav', '00060-12 15D RC REINCARNATION.wav', '00060-09 15D RC REINCARNATION.wav', '00061-10 16A RIFFS AND BEARDS.wav', '00061-01 16A RIFFS AND BEARDS.wav', '00061-02 16A RIFFS AND BEARDS.wav', '00061-04 16A RIFFS AND BEARDS.wav', '00061-03 16A RIFFS AND BEARDS.wav', '00061-07 16A RIFFS AND BEARDS.wav', '00061-05 16A RIFFS AND BEARDS.wav', '00061-08 16A RIFFS AND BEARDS.wav', '00061-12 16A RIFFS AND BEARDS.wav', '00061-11 16A RIFFS AND BEARDS.wav', '00061-09 16A RIFFS AND BEARDS.wav', '00061-06 16A RIFFS AND BEARDS.wav', '00062-07 16B FELIX MARK IV.wav', '00062-05 16B FELIX MARK IV.wav', '00062-02 16B FELIX MARK IV.wav', '00062-12 16B FELIX MARK IV.wav', '00062-09 16B FELIX MARK IV.wav', '00062-01 16B FELIX MARK IV.wav', '00062-08 16B FELIX MARK IV.wav', '00062-11 16B FELIX MARK IV.wav', '00062-10 16B FELIX MARK IV.wav', '00062-04 16B FELIX MARK IV.wav', '00062-03 16B FELIX MARK IV.wav', '00062-06 16B FELIX MARK IV.wav', '00063-03 16C FELIX JAZZ 120.wav', '00063-08 16C FELIX JAZZ 120.wav', '00063-05 16C FELIX JAZZ 120.wav', '00063-10 16C FELIX JAZZ 120.wav', '00063-06 16C FELIX JAZZ 120.wav', '00063-09 16C FELIX JAZZ 120.wav', '00063-01 16C FELIX JAZZ 120.wav', '00063-07 16C FELIX JAZZ 120.wav', '00063-11 16C FELIX JAZZ 120.wav', '00063-12 16C FELIX JAZZ 120.wav', '00063-02 16C FELIX JAZZ 120.wav', '00063-04 16C FELIX JAZZ 120.wav', '00064-05 16D FELIX DELUXE MOD.wav', '00064-11 16D FELIX DELUXE MOD.wav', '00064-09 16D FELIX DELUXE MOD.wav', '00064-01 16D FELIX DELUXE MOD.wav', '00064-07 16D FELIX DELUXE MOD.wav', '00064-03 16D FELIX DELUXE MOD.wav', '00064-08 16D FELIX DELUXE MOD.wav', '00064-06 16D FELIX DELUXE MOD.wav', '00064-02 16D FELIX DELUXE MOD.wav', '00064-12 16D FELIX DELUXE MOD.wav', '00064-04 16D FELIX DELUXE MOD.wav', '00064-10 16D FELIX DELUXE MOD.wav', '00065-05 17A FELIX ENGL.wav', '00065-12 17A FELIX ENGL.wav', '00065-02 17A FELIX ENGL.wav', '00065-06 17A FELIX ENGL.wav', '00065-07 17A FELIX ENGL.wav', '00065-11 17A FELIX ENGL.wav', '00065-01 17A FELIX ENGL.wav', '00065-08 17A FELIX ENGL.wav', '00065-04 17A FELIX ENGL.wav', '00065-03 17A FELIX ENGL.wav', '00065-10 17A FELIX ENGL.wav', '00065-09 17A FELIX ENGL.wav', '00066-03 17B SPOTLIGHTS.wav', '00066-04 17B SPOTLIGHTS.wav', '00066-08 17B SPOTLIGHTS.wav', '00066-12 17B SPOTLIGHTS.wav', '00066-02 17B SPOTLIGHTS.wav', '00066-11 17B SPOTLIGHTS.wav', '00066-01 17B SPOTLIGHTS.wav', '00066-06 17B SPOTLIGHTS.wav', '00066-05 17B SPOTLIGHTS.wav', '00066-10 17B SPOTLIGHTS.wav', '00066-09 17B SPOTLIGHTS.wav', '00066-07 17B SPOTLIGHTS.wav', '00067-09 17C BUMBLE ACOUSTIC.wav', '00067-10 17C BUMBLE ACOUSTIC.wav', '00067-07 17C BUMBLE ACOUSTIC.wav', '00067-03 17C BUMBLE ACOUSTIC.wav', '00067-06 17C BUMBLE ACOUSTIC.wav', '00067-12 17C BUMBLE ACOUSTIC.wav', '00067-02 17C BUMBLE ACOUSTIC.wav', '00067-01 17C BUMBLE ACOUSTIC.wav', '00067-04 17C BUMBLE ACOUSTIC.wav', '00067-05 17C BUMBLE ACOUSTIC.wav', '00067-08 17C BUMBLE ACOUSTIC.wav', '00067-11 17C BUMBLE ACOUSTIC.wav', '00068-07 17D BMBLFOOT PRINCE.wav', '00068-05 17D BMBLFOOT PRINCE.wav', '00068-04 17D BMBLFOOT PRINCE.wav', '00068-10 17D BMBLFOOT PRINCE.wav', '00068-03 17D BMBLFOOT PRINCE.wav', '00068-02 17D BMBLFOOT PRINCE.wav', '00068-11 17D BMBLFOOT PRINCE.wav', '00068-12 17D BMBLFOOT PRINCE.wav', '00068-06 17D BMBLFOOT PRINCE.wav', '00068-09 17D BMBLFOOT PRINCE.wav', '00068-08 17D BMBLFOOT PRINCE.wav', '00068-01 17D BMBLFOOT PRINCE.wav', '00069-10 18A SHEEHAN PEARCE.wav', '00069-07 18A SHEEHAN PEARCE.wav', '00069-08 18A SHEEHAN PEARCE.wav', '00069-09 18A SHEEHAN PEARCE.wav', '00069-12 18A SHEEHAN PEARCE.wav', '00069-02 18A SHEEHAN PEARCE.wav', '00069-03 18A SHEEHAN PEARCE.wav', '00069-11 18A SHEEHAN PEARCE.wav', '00069-04 18A SHEEHAN PEARCE.wav', '00069-05 18A SHEEHAN PEARCE.wav', '00069-01 18A SHEEHAN PEARCE.wav', '00069-06 18A SHEEHAN PEARCE.wav', '00070-11 18B SHEEHAN SVT4PRO.wav', '00070-06 18B SHEEHAN SVT4PRO.wav', '00070-05 18B SHEEHAN SVT4PRO.wav', '00070-01 18B SHEEHAN SVT4PRO.wav', '00070-04 18B SHEEHAN SVT4PRO.wav', '00070-12 18B SHEEHAN SVT4PRO.wav', '00070-03 18B SHEEHAN SVT4PRO.wav', '00070-09 18B SHEEHAN SVT4PRO.wav', '00070-10 18B SHEEHAN SVT4PRO.wav', '00070-08 18B SHEEHAN SVT4PRO.wav', '00070-07 18B SHEEHAN SVT4PRO.wav', '00070-02 18B SHEEHAN SVT4PRO.wav', '00071-06 18C THE BLUE AGAVE.wav', '00071-11 18C THE BLUE AGAVE.wav', '00071-01 18C THE BLUE AGAVE.wav', '00071-09 18C THE BLUE AGAVE.wav', '00071-10 18C THE BLUE AGAVE.wav', '00071-04 18C THE BLUE AGAVE.wav', '00071-12 18C THE BLUE AGAVE.wav', '00071-03 18C THE BLUE AGAVE.wav', '00071-08 18C THE BLUE AGAVE.wav', '00071-07 18C THE BLUE AGAVE.wav', '00071-05 18C THE BLUE AGAVE.wav', '00071-02 18C THE BLUE AGAVE.wav', '00072-02 18D BULB RHYTHM.wav', '00072-11 18D BULB RHYTHM.wav', '00072-07 18D BULB RHYTHM.wav', '00072-03 18D BULB RHYTHM.wav', '00072-05 18D BULB RHYTHM.wav', '00072-09 18D BULB RHYTHM.wav', '00072-12 18D BULB RHYTHM.wav', '00072-06 18D BULB RHYTHM.wav', '00072-10 18D BULB RHYTHM.wav', '00072-08 18D BULB RHYTHM.wav', '00072-01 18D BULB RHYTHM.wav', '00072-04 18D BULB RHYTHM.wav', '00073-11 19A BULB LEAD.wav', '00073-03 19A BULB LEAD.wav', '00073-09 19A BULB LEAD.wav', '00073-01 19A BULB LEAD.wav', '00073-02 19A BULB LEAD.wav', '00073-10 19A BULB LEAD.wav', '00073-04 19A BULB LEAD.wav', '00073-05 19A BULB LEAD.wav', '00073-06 19A BULB LEAD.wav', '00073-08 19A BULB LEAD.wav', '00073-12 19A BULB LEAD.wav', '00073-07 19A BULB LEAD.wav', '00074-08 19B BULB CLEAN.wav', '00074-02 19B BULB CLEAN.wav', '00074-04 19B BULB CLEAN.wav', '00074-05 19B BULB CLEAN.wav', '00074-07 19B BULB CLEAN.wav', '00074-12 19B BULB CLEAN.wav', '00074-03 19B BULB CLEAN.wav', '00074-10 19B BULB CLEAN.wav', '00074-06 19B BULB CLEAN.wav', '00074-09 19B BULB CLEAN.wav', '00074-11 19B BULB CLEAN.wav', '00074-01 19B BULB CLEAN.wav', '00075-06 19C BULB AMBIENT.wav', '00075-07 19C BULB AMBIENT.wav', '00075-09 19C BULB AMBIENT.wav', '00075-11 19C BULB AMBIENT.wav', '00075-03 19C BULB AMBIENT.wav', '00075-04 19C BULB AMBIENT.wav', '00075-10 19C BULB AMBIENT.wav', '00075-02 19C BULB AMBIENT.wav', '00075-12 19C BULB AMBIENT.wav', '00075-08 19C BULB AMBIENT.wav', '00075-01 19C BULB AMBIENT.wav', '00075-05 19C BULB AMBIENT.wav', '00076-11 19D EMPTY GARBAGE.wav', '00076-02 19D EMPTY GARBAGE.wav', '00076-03 19D EMPTY GARBAGE.wav', '00076-05 19D EMPTY GARBAGE.wav', '00076-04 19D EMPTY GARBAGE.wav', '00076-07 19D EMPTY GARBAGE.wav', '00076-09 19D EMPTY GARBAGE.wav', '00076-10 19D EMPTY GARBAGE.wav', '00076-12 19D EMPTY GARBAGE.wav', '00076-06 19D EMPTY GARBAGE.wav', '00076-01 19D EMPTY GARBAGE.wav', '00076-08 19D EMPTY GARBAGE.wav', '00077-12 20A ONLY GARBAGE.wav', '00077-10 20A ONLY GARBAGE.wav', '00077-09 20A ONLY GARBAGE.wav', '00077-01 20A ONLY GARBAGE.wav', '00077-07 20A ONLY GARBAGE.wav', '00077-03 20A ONLY GARBAGE.wav', '00077-04 20A ONLY GARBAGE.wav', '00077-08 20A ONLY GARBAGE.wav', '00077-02 20A ONLY GARBAGE.wav', '00077-05 20A ONLY GARBAGE.wav', '00077-11 20A ONLY GARBAGE.wav', '00077-06 20A ONLY GARBAGE.wav', '00078-12 20B GARBAGE BASS.wav', '00078-06 20B GARBAGE BASS.wav', '00078-03 20B GARBAGE BASS.wav', '00078-07 20B GARBAGE BASS.wav', '00078-08 20B GARBAGE BASS.wav', '00078-01 20B GARBAGE BASS.wav', '00078-11 20B GARBAGE BASS.wav', '00078-10 20B GARBAGE BASS.wav', '00078-02 20B GARBAGE BASS.wav', '00078-04 20B GARBAGE BASS.wav', '00078-05 20B GARBAGE BASS.wav', '00078-09 20B GARBAGE BASS.wav', '00079-09 20C BILLY KASTODON.wav', '00079-07 20C BILLY KASTODON.wav', '00079-05 20C BILLY KASTODON.wav', '00079-06 20C BILLY KASTODON.wav', '00079-04 20C BILLY KASTODON.wav', '00079-02 20C BILLY KASTODON.wav', '00079-11 20C BILLY KASTODON.wav', '00079-03 20C BILLY KASTODON.wav', '00079-01 20C BILLY KASTODON.wav', '00079-12 20C BILLY KASTODON.wav', '00079-08 20C BILLY KASTODON.wav', '00079-10 20C BILLY KASTODON.wav', '00080-10 20D THIS IS THE END.wav', '00080-08 20D THIS IS THE END.wav', '00080-11 20D THIS IS THE END.wav', '00080-07 20D THIS IS THE END.wav', '00080-06 20D THIS IS THE END.wav', '00080-04 20D THIS IS THE END.wav', '00080-03 20D THIS IS THE END.wav', '00080-09 20D THIS IS THE END.wav', '00080-12 20D THIS IS THE END.wav', '00080-05 20D THIS IS THE END.wav', '00080-01 20D THIS IS THE END.wav', '00080-02 20D THIS IS THE END.wav', '00000-05 DI.wav', '00000-12 DI.wav', '00000-03 DI.wav', '00000-08 DI.wav', '00000-07 DI.wav', '00000-06 DI.wav', '00000-01 DI.wav', '00000-09 DI.wav', '00000-10 DI.wav', '00000-11 DI.wav', '00000-04 DI.wav', '00000-02 DI.wav']

In [5]:

for file in files:

    if re.match(AUDIO_REGEX, file) and number_of_clips > 0:
        print(file)
        number_of_clips -= 1
        y, sr = librosa.load(os.path.join(AUDIO_FILES_DIR, file), sr=SAMPLE_RATE, mono=MONO)
        duration = librosa.get_duration(y=y, sr=sr)
        window_id = 0
        while True:
            if (window_id) * WINDOW_DURATION >= duration:
                break
            
            offset = window_id * WINDOW_DURATION
            loader = Loader(SAMPLE_RATE, offset, min(WINDOW_DURATION, duration - offset), WINDOW_DURATION, MONO)
            preprocessing_pipeline.loader = loader
            visualizer = Visualizer(SPECTROGRAMS_IMG_SAVE_DIR, FRAME_SIZE, HOP_LENGTH, offset)
            preprocessing_pipeline.visualizer = visualizer

            preprocessing_pipeline.process_file(AUDIO_FILES_DIR, file, window_id, save_flg=True, visualize_flg=True, external_features_flg=False)
            window_id += 1

00017-07 05A Placater Clean.wav
	00017-07-00 05A Placater Clean.wav
	00017-07-01 05A Placater Clean.wav
	00017-07-02 05A Placater Clean.wav
	00017-07-03 05A Placater Clean.wav
	00017-07-04 05A Placater Clean.wav
	00017-07-05 05A Placater Clean.wav
	00017-07-06 05A Placater Clean.wav
	00017-07-07 05A Placater Clean.wav
	00017-07-08 05A Placater Clean.wav
	00017-07-09 05A Placater Clean.wav
	00017-07-10 05A Placater Clean.wav
00017-06 05A Placater Clean.wav
	00017-06-00 05A Placater Clean.wav
	00017-06-01 05A Placater Clean.wav
	00017-06-02 05A Placater Clean.wav
	00017-06-03 05A Placater Clean.wav
	00017-06-04 05A Placater Clean.wav
	00017-06-05 05A Placater Clean.wav
	00017-06-06 05A Placater Clean.wav
	00017-06-07 05A Placater Clean.wav
	00017-06-08 05A Placater Clean.wav
	00017-06-09 05A Placater Clean.wav
	00017-06-10 05A Placater Clean.wav
	00017-06-11 05A Placater Clean.wav
	00017-06-12 05A Placater Clean.wav
	00017-06-13 05A Placater Clean.wav
00017-12 05A Placater Clean.wav
	000

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:111: RuntimeWarning: invalid value encountered in true_divide


	00032-08-13 08D Line 6 Badonk.wav
Visualizer: Null spectrogram for file: 00032-08-13 08D Line 6 Badonk.wav
00032-01 08D Line 6 Badonk.wav
	00032-01-00 08D Line 6 Badonk.wav
	00032-01-01 08D Line 6 Badonk.wav
	00032-01-02 08D Line 6 Badonk.wav
	00032-01-03 08D Line 6 Badonk.wav
	00032-01-04 08D Line 6 Badonk.wav
	00032-01-05 08D Line 6 Badonk.wav
	00032-01-06 08D Line 6 Badonk.wav
	00032-01-07 08D Line 6 Badonk.wav
	00032-01-08 08D Line 6 Badonk.wav
	00032-01-09 08D Line 6 Badonk.wav
	00032-01-10 08D Line 6 Badonk.wav
	00032-01-11 08D Line 6 Badonk.wav
	00032-01-12 08D Line 6 Badonk.wav
Visualizer: Null spectrogram for file: 00032-01-12 08D Line 6 Badonk.wav
00032-09 08D Line 6 Badonk.wav
	00032-09-00 08D Line 6 Badonk.wav
	00032-09-01 08D Line 6 Badonk.wav
	00032-09-02 08D Line 6 Badonk.wav
	00032-09-03 08D Line 6 Badonk.wav
	00032-09-04 08D Line 6 Badonk.wav
	00032-09-05 08D Line 6 Badonk.wav
	00032-09-06 08D Line 6 Badonk.wav
	00032-09-07 08D Line 6 Badonk.wav
	00032-09-08 08D Line 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:160: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


	00054-12-01 14B DUSTED.wav
	00054-12-02 14B DUSTED.wav
	00054-12-03 14B DUSTED.wav
	00054-12-04 14B DUSTED.wav
	00054-12-05 14B DUSTED.wav
	00054-12-06 14B DUSTED.wav
	00054-12-07 14B DUSTED.wav
	00054-12-08 14B DUSTED.wav
	00054-12-09 14B DUSTED.wav
	00054-12-10 14B DUSTED.wav
	00054-12-11 14B DUSTED.wav
	00054-12-12 14B DUSTED.wav
	00054-12-13 14B DUSTED.wav
	00054-12-14 14B DUSTED.wav
00054-07 14B DUSTED.wav
	00054-07-00 14B DUSTED.wav
	00054-07-01 14B DUSTED.wav
	00054-07-02 14B DUSTED.wav
	00054-07-03 14B DUSTED.wav
	00054-07-04 14B DUSTED.wav
	00054-07-05 14B DUSTED.wav
	00054-07-06 14B DUSTED.wav
	00054-07-07 14B DUSTED.wav
	00054-07-08 14B DUSTED.wav
	00054-07-09 14B DUSTED.wav
	00054-07-10 14B DUSTED.wav
00054-08 14B DUSTED.wav
	00054-08-00 14B DUSTED.wav
	00054-08-01 14B DUSTED.wav
	00054-08-02 14B DUSTED.wav
	00054-08-03 14B DUSTED.wav
	00054-08-04 14B DUSTED.wav
	00054-08-05 14B DUSTED.wav
	00054-08-06 14B DUSTED.wav
	00054-08-07 14B DUSTED.wav
	00054-08-08 14B DUSTED.wav
